In [1]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn
from data_generator_glad import *
import pickle as pkl
import warnings
warnings.filterwarnings("ignore")
print("import package finished.")

2.3
import package finished.


In [2]:
K_train = 10 # 10
M = 100
N = 20 # number of features
graph_type = "BA"
SAMPLE_BATCHES = 10
w_min = -1
w_max = 1
SIGNS = 0
K_test = 100 # 100
prob = 0.1
MAX_DEG = 50
K_valid = 10
edge_type = "lognormal"
scale = True
ee = 1e-4
data_path = "./data/syn/Ktrain{}_Ktest{}_M{}_N{}_prob{}.pkl".format(K_train, K_test, M, N, prob)
train_data_path = "data/syn/dataset_{}_{}_{}nodes_train_{}_{}.pickle".format(graph_type, edge_type, N, scale, ee)
test_data_path = "data/syn/dataset_{}_{}_{}nodes_test_{}_{}.pickle".format(graph_type, edge_type, N, scale, ee)
val_data_path = "data/syn/dataset_{}_{}_{}nodes_val_{}_{}.pickle".format(graph_type, edge_type, N, scale, ee)

with open(train_data_path, 'rb') as trainf:
    train_data_ = pkl.load(trainf)
train_data = [np.array(train_data_['p']), np.array(train_data_['cov'])]
print(train_data[0].shape)

with open(test_data_path, 'rb') as testf:
    test_data_ = pkl.load(testf)
test_data = [np.array(test_data_['p']), np.array(test_data_['cov'])]
print(test_data[0].shape)

with open(val_data_path, 'rb') as valf:
    val_data_ = pkl.load(valf)
val_data = [np.array(val_data_['p']), np.array(val_data_['cov'])]
print(val_data[0].shape)

(4000, 20, 20)
(64, 20, 20)
(1000, 20, 20)


In [3]:
from unrolled_model import *
from torch.optim.lr_scheduler import LambdaLR, StepLR, MultiStepLR, ExponentialLR, ReduceLROnPlateau
import random
import binary_metrics as metrics
print("import training needed packages")

import training needed packages


In [4]:
TRAIN = True
USE_CUDA = True
L = 30
rho_init = 1
lambda_init = 1
theta_init_offset = 1e-2
gamma_init = 0.1
nF = 3
H = 3
init_lr = 0.001
use_optimizer = 'adam'
batch_size = 10
train_epochs = 5
INIT_DIAG = 0
lossBCE = 0
loss_signed = 0
lossL1 = 0
model_path = "M{}_N{}_Ktrain{}_gt{}_prob{}_bs{}_L{}_lr{}_ee{}.pth".format(M, N, K_train, graph_type, prob, batch_size, L, init_lr, ee)

In [5]:
def train_glasso(data, valid_data=[]):  # tied lista
    #    torch.set_grad_enabled(True)
    print('training GLASSO')
    theta, S = data
    #    theta = theta[0]
    if len(valid_data) > 0:
        valid_theta, valid_S = valid_data
        valid_theta_true = convert_to_torch(valid_theta, TESTING_FLAG=True, USE_CUDA=USE_CUDA)
        valid_S = convert_to_torch(valid_S, TESTING_FLAG=True, USE_CUDA=USE_CUDA)
    # theta -> K_train x N x N (Matrix)
    # S -> K_train x N x N (observed vector)
    # train using ALISTA style training.
    # model = threshold_NN_lambda_single_model(L, rho_init, lambda_init, theta_init_offset, gamma_init, N, nF, H, USE_CUDA=USE_CUDA)
    model = threshold_NN_lambda_unrolled_model(L, rho_init, lambda_init, theta_init_offset, gamma_init, N, nF, H, USE_CUDA=USE_CUDA)

    model.train()
    theta_true = convert_to_torch(theta, TESTING_FLAG=True, USE_CUDA=USE_CUDA)
    S = convert_to_torch(S, TESTING_FLAG=True, USE_CUDA=USE_CUDA)

    zero = torch.Tensor([0])  # .type(self.dtype)

    #    print('check: theta ', theta_init.shape)
    #    print('true: ', theta_true)
    print('parameters to be learned')
    for name, param in model.named_parameters():
        print(name, param.shape, param.requires_grad)
    dtype = torch.FloatTensor
    if USE_CUDA:
        model = model.cuda()
        zero = zero.cuda()
        dtype = torch.cuda.FloatTensor

    lr = init_lr
    if use_optimizer == 'adadelta':
        optimizer = torch.optim.Adadelta(model.parameters(), lr=lr, rho=0.9, eps=1e-06,
                                         weight_decay=0)  # LR range = 5 ->
    elif use_optimizer == 'rms':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.25,
                                        centered=False)
    elif use_optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, dampening=0, weight_decay=0,
                                    nesterov=False)
    elif use_optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    else:
        print('Optimizer not found!')
    # scheduler = MultiStepLR(optimizer, milestones=[1000], gamma=0.5)
    scheduler = MultiStepLR(optimizer, milestones=[10, 15, 20, 25, 100, 200], gamma=0.25)
    # scheduler = MultiStepLR(optimizer, milestones=[10, 15, 20, 100, 200], gamma=0.25)
    # scheduler = MultiStepLR(optimizer, milestones=[10, 20, 30, 200], gamma=0.5)
    # criterion = nn.MSELoss(reduction="sum")  # input, target
    criterion = nn.MSELoss()  # input, target
    criterion_L1 = nn.L1Loss()
    m_sig = nn.Sigmoid()
    criterionBCE = nn.BCELoss()

    # batch size is fixed
    #    num_batches = int(args.K_train/args.batch_size)
    #    if args.SAMPLE_BATCHES > 0:
    #        num_batches = int(args.K_train*args.SAMPLE_BATCHES/args.batch_size)

    num_batches = int(len(S) / batch_size)

    #    if args.K_train >= 10:
    #        args.batch_size = 10
    # best_shd_model = model
    best_valid_shd, best_valid_ps, best_valid_nmse = np.inf, -1 * np.inf, np.inf
    EARLY_STOP = False
    for epoch in range(train_epochs):  # 1 epoch is expected to go through complete data
        scheduler.step()
        #        if epoch%1==0:
        #            for param_group in optimizer.param_groups:
        #                print('epoch: ', epoch, ' lr ', param_group['lr'])
        epoch_loss = []
        frob_loss = []
        duality_gap = []
        mse_binary_loss = []
        bce_loss = []
        if EARLY_STOP:
            break
        #        print('ecpohc ', epoch)
        for batch_num in range(num_batches):  # processing batchwise
            optimizer.zero_grad()
            # resetting the loss to zero
            loss = torch.Tensor([0]).type(dtype)
            # Get a batch
            # ridx = random.sample(list(range(args.K_train)), args.batch_size)
            ridx = random.sample(list(range(len(S))), batch_size)
            Sb = S[ridx]  # [0]
            #            print('errr train check : ', batch_num, theta_true, Sb, theta_true.expand_as(Sb))

            if INIT_DIAG == 1:
                # print(' extract batchwise diagonals, add offset and take inverse')
                batch_diags = 1 / (torch.diagonal(Sb, offset=0, dim1=-2, dim2=-1) + model.theta_init_offset)
                theta_init = torch.diag_embed(batch_diags)
            else:
                # print('***************** (S+theta_offset*I)^-1 is used')
                theta_init = torch.inverse(
                    Sb + model.theta_init_offset * torch.eye(Sb.shape[-1]).expand_as(Sb).type_as(Sb))


            # theta_pred = S_inv[r_idx]
            # ll = torch.cholesky(theta_init[ridx])#(theta_pred) # lower triangular
            # ll = my_cholesky(theta_init[ridx][0])#(theta_pred) # lower triangular
            # ll = batch_cholesky(theta_init[ridx])#(theta_pred) # lower triangular
            theta_pred = theta_init  # [ridx]
            # theta_pred = theta_init[ridx]
            # theta_pred.requires_grad = True
            # Sb = S[ridx][0]
            # step_size = get_init_step_size(theta_init[ridx])
            identity_mat = torch.eye(Sb.shape[-1]).expand_as(Sb)
            if USE_CUDA == True:
                identity_mat = identity_mat.cuda()
            # print('ERRR check: ', theta_pred.shape, get_frobenius_norm(theta_pred), get_frobenius_norm(theta_pred).shape)
            # lambda_k = model.lambda_f(get_frobenius_norm(theta_pred))
            lambda_k = model.lambda_forward(zero + lambda_init, zero, k=0)
            for k in range(L):
                #                print('itr = ', itr, theta_pred)#, theta_true[ridx])
                # step 1 : AM
                b = 1.0 / lambda_k * Sb - theta_pred
                b2_4ac = torch.matmul(b.transpose(-1, -2), b) + 4.0 / lambda_k * identity_mat
                sqrt_term = batch_matrix_sqrt(b2_4ac)
                theta_k1 = 1.0 / 2 * (-1 * b + sqrt_term)
                """
                # extract the diagonals of the matrices
                theta_diag = torch.diag_embed(torch.diagonal(theta_k1, offset=0, dim1=-2, dim2=-1))
                # soft threshold on remaining entries
                theta_pred = model.eta_forward(theta_k1-theta_diag, k)
                # add the diagonals
                theta_pred = theta_pred + theta_diag
                """
                # softthresholding on all the entries
                # theta_pred = model.eta_forward(theta_k1, k)

                # if MODEL_type == 'th':
                #     # soft thresholding + eigenvalue correctness term
                #     theta_pred = model.eta_forward(theta_k1, k) + torch.max(model.gamma_c[k],
                #                                                             zero + 1e-2) * identity_mat
                # elif MODEL_type == 'th_NN':
                theta_pred = model.eta_forward(theta_k1, Sb, k, theta_pred)  #
                # update the lambda
                lambda_k = model.lambda_forward(torch.Tensor([get_frobenius_norm(theta_pred - theta_k1)]).type(dtype),
                                                lambda_k, k)
                # accumulating loss
                #                print('k= ', k, ' lambda_value ', lambda_k, get_frobenius_norm(theta_pred-theta_k1))

                loss += criterion(theta_pred, theta_true[ridx]) / L
                # loss += criterion(theta_pred, theta_true.expand_as(theta_pred))/args.L

            #            print('k= ', k, ' lambda_value ', lambda_k)
            # print('thetapred: ', theta_pred, check_sym(theta_pred[0].data.cpu().numpy()))
            # delta = batch_duality_gap(theta_pred, Sb, model.rho)
            # NOTE: ******* IMP: Change thissss!@!! ****************************
            delta = torch.ones([1]) * -1

            loss += criterion(theta_pred, theta_true[ridx]) / L

            # loss += criterion(theta_pred, theta_true.expand_as(theta_pred))/args.L

            lossf = get_frobenius_norm(theta_pred - theta_true[ridx])
#             lossgmse = get_frobenius_norm_except_diag(theta_pred - theta_true[ridx])
            # total_loss = loss #+ delta
            # total_loss = lossB #+ loss #+ delta
            # total_loss = loss + lossB+ delta + lossBCE
            #            total_loss = lossBCE
            # total_loss = delta
            if lossBCE == 1:  # binary cross entropy
                total_loss = lossBCE
            elif loss_signed == 1:  # signed loss
                # total_loss = criterion(torch.sign(theta_pred), torch.sign(theta_true.expand_as(theta_pred)))
                total_loss = criterion(theta_pred, torch.sign(theta_true.expand_as(theta_pred)))
            elif lossL1 == 1:  # signed loss
                total_loss = criterion_L1(theta_pred, theta_true.expand_as(theta_pred))
            else:  # frobenius norm
                total_loss = loss
                # total_loss = lossf
            #            total_loss.requires_grad = True
            #            print('err: ', total_loss, total_loss.requires_grad)

            lv = loss.data.cpu().numpy()
            if lv <= 1e-7:  # loss value
                print('Early stopping as loss = ', lv)
                EARLY_STOP = True
                break

            total_loss.backward()
            # delta.backward()

            #            for name, param in model.named_parameters():
            #                print('befoer: ', name, param)
            optimizer.step()

            #            for name, param in model.named_parameters():
            #                print('after: ', name, param)

            #            mse_binary_loss.append(lossB.data.cpu().numpy())
            #            bce_loss.append(lossBCE.data.cpu().numpy())
            #            duality_gap.append(delta.data.cpu().numpy())
            frob_loss.append(lossf.data.cpu().numpy())
            epoch_loss.append(loss.data.cpu().numpy())
        if epoch % 2 == 0 and EARLY_STOP == False:
            # print(len(epoch_loss))
            # print('loss_summary: MSE: ', sum(epoch_loss)/len(epoch_loss), ' Mean Frobenius loss: ',sum(frob_loss)/len(frob_loss), ' MSE_binary loss: ', sum(mse_binary_loss)/len(mse_binary_loss), 'BCE_loss: ', sum(bce_loss)/len(bce_loss), 'duality gap = ', sum(duality_gap)/len(duality_gap))
            print('loss_summary: MSE: ', sum(epoch_loss)/len(epoch_loss), ' Mean Frobenius loss: ',sum(frob_loss)/len(frob_loss))
            if lossBCE == 1:
                print(epoch, sum(epoch_loss) / len(epoch_loss), sum(bce_loss) / len(bce_loss))
            else:
                print('loss_values: ', epoch, sum(epoch_loss) / len(epoch_loss))  # , sum(duality_gap)/len(duality_gap))
        # Checking the results on valid data and updating the best model
        if len(valid_data) > 0:
            # get the SHD on the valid data and the train data
            # curr_valid_shd, curr_valid_nmse = glasso_predict(model, valid_data)
            curr_valid_ps_mean, curr_valid_ps_std, \
            curr_valid_aps_mean, curr_valid_aps_std, \
            curr_valid_aucs_mean, curr_valid_aucs_std, \
            curr_valid_nmse, \
            curr_valid_gmse, curr_valid_gmse_m, curr_valid_gmse_h = glasso_predict(model, valid_data)
            curr_train_ps_mean, curr_train_ps_std, \
            curr_train_aps_mean, curr_train_aps_std, \
            curr_train_aucs_mean, curr_train_aucs_std, \
            curr_train_nmse, \
            curr_train_gmse, curr_train_gmse_m, curr_train_gmse_h = glasso_predict(model, data)
            print('valid/train: ps %0.4f/%0.4f aps %0.4f/%0.4f aucs %0.4f/%0.4f nmse %0.4f/%0.4f' % (
            curr_valid_ps_mean, curr_train_ps_mean, curr_valid_aps_mean, curr_train_aps_mean,
            curr_valid_aucs_mean, curr_train_aucs_mean, curr_valid_nmse, curr_train_nmse))
            #            if curr_valid_shd <= best_valid_shd:
            if curr_valid_ps_mean >= best_valid_ps:
                print('epoch = ', epoch, ' Updating the best ps model with valid ps = ', curr_valid_ps_mean)
                best_ps_model = copy.deepcopy(model)
                best_valid_ps = curr_valid_ps_mean

            # if curr_valid_shd <= best_valid_shd:
            #     print('epoch = ', epoch, ' Updating the best shd model with valid shd = ', curr_valid_shd)
            #     best_shd_model = copy.deepcopy(model)
            #     best_valid_shd = curr_valid_shd

            if curr_valid_nmse <= best_valid_nmse:
                print('epoch = ', epoch, ' Updating the best nmse model with valid nmse = ', curr_valid_nmse)
                best_nmse_model = copy.deepcopy(model)
                best_valid_nmse = curr_valid_nmse
                print('epoch = ', epoch, ' Updating the best nmse model with valid gmse = ', curr_valid_gmse)
                print('epoch = ', epoch, ' Updating the best nmse model with valid gmse except diag = {}+-{}'.format(curr_valid_gmse_m, curr_valid_gmse_h))
            model.train()
            print('loss_summary:: epoch: ', epoch, ' loss: ', sum(epoch_loss)/len(epoch_loss))#, ' NMSE loss: ', 10*np.log10( (np.sum(np.array(epoch_loss)))/(len(epoch_loss)*E_norm_xtrue)))
    #    print('ans: ', theta_pred)
    #    print('true: ', theta_true)
    for name, param in model.named_parameters():
        print(name, param)
    # return best_ps_model # model
    # return best_nmse_model, best_shd_model, best_ps_model  # model
    return best_nmse_model, best_ps_model

In [6]:
def glasso_predict(model, data, flagP=False, SAVE_GRAPH=False, eM=0, name='', mn=''):
    with torch.no_grad():
        print('Running unrolled ADMM predict')
        # predict as a complete batch?
        model.eval()
        criterion = nn.MSELoss()  # input, target
        m_sig = nn.Sigmoid()
        criterionBCE = nn.BCELoss()
        theta, S = data
        #    theta = theta[0]
        # theta -> K_train x N x N (Matrix)
        # S -> K_train x N x N (observed vector)
        # theta_true = convert_to_torch(theta, TESTING_FLAG=True, USE_CUDA=False)
        theta_true = convert_to_torch(theta, TESTING_FLAG=True, USE_CUDA=USE_CUDA)
        S = convert_to_torch(S, TESTING_FLAG=True, USE_CUDA=USE_CUDA)

        zero = torch.Tensor([0])  # .type(self.dtype)
        dtype = torch.FloatTensor
        if USE_CUDA == True:
            zero = zero.cuda()
            model = model.cuda()
            dtype = torch.cuda.FloatTensor

        # batch size is fixed for testing as 1
        batch_size = 1
        print('CEHCKK: Total graphs = ', len(S))
        num_batches = int(len(S) / batch_size)
        #    print('num batches: ', num_batches)
        epoch_loss = []
        mse_binary_loss = []
        bce_loss = []
        frob_loss = []
        duality_gap = []
        ans = []
        if flagP:
            res_conv = {}
            for k in range(L + 1):
                res_conv[k] = []
        #        print('ITR, conv.loss, obj_val_pred, obj_val_true_model_rho, obj_val_pred_args_rho')#, theta_pred)

        res = []
        gmse_res = []
        for batch_num in range(num_batches):  # processing batchwise
            # Get a batch
            # ll = my_cholesky(theta_init[ridx][0])#(theta_pred) # lower triangular
            # theta_pred = theta_init[batch_num*batch_size: (batch_num+1)*batch_size] #(theta_pred) # lower triangular
            # theta_true_b = theta_true[batch_num*batch_size: (batch_num+1)*batch_size]
            theta_true_b = theta_true[batch_num * batch_size: (batch_num + 1) * batch_size]
            Sb = S[batch_num * batch_size: (batch_num + 1) * batch_size]  # [0]
            identity_mat = torch.eye(Sb.shape[-1]).expand_as(Sb)
            if USE_CUDA == True:
                identity_mat = identity_mat.cuda()
                Sb = Sb.cuda()
            #    theta_pred = theta_pred.cuda()
            #    theta_true_b = theta_true_b.cuda()

            if INIT_DIAG == 1:
                # print(' extract batchwise diagonals, add offset and take inverse')
                batch_diags = 1 / (torch.diagonal(Sb, offset=0, dim1=-2, dim2=-1) + model.theta_init_offset)
                theta_pred = torch.diag_embed(batch_diags)
            else:
                # print('***************** (S+theta_offset*I)^-1 is used')
                theta_pred = torch.inverse(
                    Sb + model.theta_init_offset * torch.eye(Sb.shape[-1]).expand_as(Sb).type_as(Sb))

            # lambda_k = model.lambda_f(get_frobenius_norm(theta_pred))
            lambda_k = model.lambda_forward(zero + lambda_init, zero, k=0)

            #        if flagP:
            #            print('ITR, conv.loss, obj_val_pred, obj_val_true_model_rho, obj_val_pred_args_rho')#, theta_pred)
            for k in range(L):
                #            start = time.time()
                if flagP:
                    theta_pred_diag = torch.diag_embed(torch.diagonal(theta_pred[0], offset=0, dim1=-2, dim2=-1))
                    # theta_true_b_diag = torch.diag_embed(torch.diagonal(theta_true_b[0], offset=0, dim1=-2, dim2=-1))
                    theta_true_b_diag = torch.diag_embed(torch.diagonal(theta_true_b, offset=0, dim1=-2, dim2=-1))
                    # if MODEL_type == 'th':
                    #     cv_loss, cv_loss_off_diag, obj_pred, obj_true_rho, obj_true_orig = get_convergence_loss(
                    #         theta_pred[0], theta_true_b), get_convergence_loss(theta_pred[0] - theta_pred_diag,
                    #                                                            theta_true_b - theta_true_b_diag), get_obj_val(
                    #         theta_pred[0], Sb[0], model.rho_l1[k]), get_obj_val(theta_true_b, Sb[0],
                    #                                                             model.rho_l1[k]), get_obj_val(
                    #         theta_true_b, Sb[0], rho_init)
                    #     res_conv[k].append([cv_loss, obj_pred, obj_true_rho, obj_true_orig, cv_loss_off_diag])
                    # elif MODEL_type == 'th_NN':
                    cv_loss, cv_loss_off_diag = get_convergence_loss(theta_pred[0],
                                                                         theta_true_b), -1  # get_convergence_loss(theta_pred[0]-theta_pred_diag, theta_true_b-theta_true_b_diag)
                    res_conv[k].append([cv_loss, cv_loss_off_diag])
                # step 1 : AM
                b = 1.0 / lambda_k * Sb - theta_pred
                b2_4ac = torch.matmul(b.transpose(-1, -2), b) + 4.0 / lambda_k * identity_mat
                sqrt_term = batch_matrix_sqrt(b2_4ac)
                theta_k1 = 1.0 / 2 * (-1 * b + sqrt_term)

                # step 2 : AM
                """
                # extract the diagonals of the matrices
                theta_diag = torch.diag_embed(torch.diagonal(theta_k1, offset=0, dim1=-2, dim2=-1))
                # soft threshold on remaining entries
                theta_pred = model.eta_forward(theta_k1-theta_diag, k)
                # add the diagonals
                theta_pred = theta_pred + theta_diag
                """
                # soft thresholding on all the entries
                # theta_pred = model.eta_forward(theta_k1, k)
                # if MODEL_type == 'th':
                #     # soft thresholding + eigenvalue correctness term
                #     theta_pred = model.eta_forward(theta_k1, k) + torch.max(model.gamma_c[k],
                #                                                             zero + 1e-2) * identity_mat
                # elif MODEL_type == 'th_NN':
                    # theta_pred = model.eta_forward(theta_k1, Sb, k) + torch.max(model.gamma_c[k], zero+1e-2) * identity_mat
                    # theta_pred = model.eta_forward(theta_k1, Sb, k, theta_pred) + torch.max(model.gamma_c[k], zero+1e-2) * identity_mat
                theta_pred = model.eta_forward(theta_k1, Sb, k,
                                                   theta_pred)  # + torch.max(model.gamma_c[k], zero+1e-2) * identity_mat

                # updating lambda
                lambda_k = model.lambda_forward(torch.Tensor([get_frobenius_norm(theta_pred - theta_k1)]).type(dtype),
                                                lambda_k, k)
                # lambda_k = model.lambda_f(get_frobenius_norm(theta_pred-theta_k1))
            #            print('k= ', k, ' lambda_value ', lambda_k, get_frobenius_norm(theta_pred-theta_k1))
            #            stop = time.time()
            #            print('Walltimes: ', k, stop-start)
            #        br
            if flagP:
                theta_pred_diag = torch.diag_embed(torch.diagonal(theta_pred[0], offset=0, dim1=-2, dim2=-1))
                # Getting the final predicted convergence loss
                if torch.min(torch.eig(theta_pred[0])[0][:, 0]) == 0:
                    adjust_eval_identity = torch.eye(theta_pred.shape[-1]).expand_as(theta_pred[0]).type_as(
                        theta_pred[0])
                    print('Adjusting the minimum eigenvalue to 1, SHOULD NOT BE CALLED AFTER THE GAMMA ADDITION!')
                    theta_pred[0] += adjust_eval_identity  # change the eigenval to 1
                # if MODEL_type == 'th':
                #     cv_loss, cv_loss_off_diag, obj_pred, obj_true_rho, obj_true_orig = get_convergence_loss(
                #         theta_pred[0], theta_true_b), get_convergence_loss(theta_pred[0] - theta_pred_diag,
                #                                                            theta_true_b - theta_true_b_diag), get_obj_val(
                #         theta_pred[0], Sb[0], model.rho_l1[k]), get_obj_val(theta_true_b, Sb[0],
                #                                                             model.rho_l1[k]), get_obj_val(theta_true_b,
                #                                                                                           Sb[0],
                #                                                                                           args.rho_init)
                #     res_conv[k + 1].append([cv_loss, obj_pred, obj_true_rho, obj_true_orig, cv_loss_off_diag])
                # elif MODEL_type == 'th_NN':
                cv_loss, cv_loss_off_diag = get_convergence_loss(theta_pred[0],
                                                                     theta_true_b), -1  # get_convergence_loss(theta_pred[0]-theta_pred_diag, theta_true_b-theta_true_b_diag)
                res_conv[k + 1].append([cv_loss, cv_loss_off_diag])

            if Plot:
                print("=====================")

                plt.figure()
                seaborn.heatmap(theta_pred[0][:100,:100].cpu().detach().numpy(),cmap="pink_r",vmax=1)
                plt.show()

                plt.figure()
                seaborn.heatmap(theta_true_b[0][:100,:100].cpu().detach().numpy(),cmap="pink_r",vmax=1)
                plt.show()

            final_nmse, final_gmse, n, d, final_gmse_exp_diag, n1, d1 = get_convergence_loss(theta_pred, theta_true_b)
            theta_pred = theta_pred.data.cpu().numpy()
            #        theta_true_b = theta_true_b[0].data.cpu().numpy()
            theta_true_b = theta_true_b.data.cpu().numpy()

            FDRs, TPRs, FPRs, SHDs, Ts, Ps, precisions, recalls, \
            F_betas, auprs, aucs = metrics.report_metrics(theta_true_b, theta_pred, batch_size=batch_size)
            cond_theta_pred, cond_theta_true_b = np.linalg.cond(theta_pred), -1  # np.linalg.cond(theta_true_b)
            res.append([FDRs, TPRs, FPRs, SHDs, Ts, Ps, precisions, recalls, F_betas, auprs, aucs])
            gmse_res.append(final_gmse_exp_diag)

        res_mean = np.mean(np.array(res), 0)
        res_std = np.std(np.array(res), 0)
        res_mean = ["%.5f" % x for x in res_mean]
        res_std = ["%.5f" % x for x in res_std]

        if flagP:
            print('Structure learning Metrics')
            print('Average result over test graphs')
            # print('fdr, tpr, fpr, shd, nnz, nnz_true, np.linalg.cond(theta_pred), np.linalg.cond(theta_true)')
            print('FDRs, TPRs, FPRs, SHDs, Ts, Ps, precisions, recalls, F_betas, auprs, aucs')
            print(*sum(list(map(list, zip(res_mean, res_std))), []), sep=', ')

            # print('ITR, conv.loss, obj_val_pred, obj_val_true_model_rho, obj_val_pred_args_rho')#, theta_pred)
#             print(
#                 'ITR, conv_loss"ecoli_M"+str(eM), obj_val_pred, obj_val_true_model_rho, obj_val_pred_args_rho, conv_loss_off_diag')  # , theta_pred)
#             for i in res_conv:
#                 mean_vec = ["%.3f" % x for x in np.mean(np.array(res_conv[i].cpu().detach()), 0)]
#                 std_vec = ["%.3f" % x for x in np.std(np.array(res_conv[i].cpu().detach()), 0)]
#                 print(i, *sum(list(map(list, zip(mean_vec, std_vec))), []), sep=', ')
        #            print(i, np.mean(res_conv[i], 0), np.std(res_conv[i], 0))

        if SAVE_GRAPH:
            x = np.where(theta_pred > 0, 1, 0)
            A = np.matrix(x - np.eye(x.shape[0]))
            G = nx.from_numpy_matrix(A)
            fig = plt.figure(figsize=(15, 15))
            mapping = {n1: n2 for n1, n2 in zip(G.nodes(), mn.nodes)}
            G = nx.relabel_nodes(G, mapping)
            # nx.draw_networkx(G, pos=nx.spring_layout(G), with_labels = True)
            # nx.draw_networkx(G, pos=nx.circular_layout(G), with_labels = True)
            nx.draw_networkx(G, pos=nx.shell_layout(G), with_labels=True)
            plt.savefig(name + '_' + str(eM) + ".pdf", bbox_inches='tight')
            nx.draw_networkx(mn.G_true, pos=nx.shell_layout(G), with_labels=True)
            plt.savefig(name + '_true_' + str(eM) + ".pdf", bbox_inches='tight')
            # saving the graph
            # nx.write_gpickle(G, name+'_'+str(eM)+'.gpickle')
            nx.write_adjlist(G, name + '_' + str(eM) + '.adjlist')

        gmsem, gmseh, _, __ = metrics.mean_confidence_interval(gmse_res)
        # pr, aps, auc
        return np.float(res_mean[5]), np.float(res_std[5]), \
               np.float(res_mean[9]), np.float(res_std[9]), \
               np.float(res_mean[10]), np.float(res_std[10]), \
               final_nmse, final_gmse, gmsem, gmseh  # The PS mean value, final NMSE obtained

In [7]:

Plot = False
if TRAIN == True:
    print('Training the glasso model')
    print('check: ', train_data[0].shape, train_data[1].shape)
    nmse_model, ps_model = train_glasso(train_data, val_data)

model = nmse_model  # shd_model # or nmse_model

torch.save(nmse_model.state_dict(), model_path)

print('TIMING check:')
glasso_predict(model, train_data)

Training the glasso model
check:  (4000, 20, 20) (4000, 20, 20)
training GLASSO
shifting to cuda
CHECK RHO INITIAL:  Parameter containing:
tensor([[-0.3543, -0.3788,  0.0514],
        [-0.4119, -0.0413, -0.5542],
        [-0.2788, -0.3570,  0.5541]], device='cuda:0', requires_grad=True)
parameters to be learned
theta_init_offset torch.Size([1]) True
rho_l1.0.0.weight torch.Size([3, 3]) True
rho_l1.0.0.bias torch.Size([3]) True
rho_l1.0.2.weight torch.Size([3, 3]) True
rho_l1.0.2.bias torch.Size([3]) True
rho_l1.0.4.weight torch.Size([3, 3]) True
rho_l1.0.4.bias torch.Size([3]) True
rho_l1.0.6.weight torch.Size([3, 3]) True
rho_l1.0.6.bias torch.Size([3]) True
rho_l1.0.8.weight torch.Size([3, 3]) True
rho_l1.0.8.bias torch.Size([3]) True
rho_l1.0.10.weight torch.Size([1, 3]) True
rho_l1.0.10.bias torch.Size([1]) True
rho_l1.1.0.weight torch.Size([3, 3]) True
rho_l1.1.0.bias torch.Size([3]) True
rho_l1.1.2.weight torch.Size([3, 3]) True
rho_l1.1.2.bias torch.Size([3]) True
rho_l1.1.4.wei

lambda_f.14.4.weight torch.Size([3, 3]) True
lambda_f.14.4.bias torch.Size([3]) True
lambda_f.14.6.weight torch.Size([1, 3]) True
lambda_f.14.6.bias torch.Size([1]) True
lambda_f.15.0.weight torch.Size([3, 2]) True
lambda_f.15.0.bias torch.Size([3]) True
lambda_f.15.2.weight torch.Size([3, 3]) True
lambda_f.15.2.bias torch.Size([3]) True
lambda_f.15.4.weight torch.Size([3, 3]) True
lambda_f.15.4.bias torch.Size([3]) True
lambda_f.15.6.weight torch.Size([1, 3]) True
lambda_f.15.6.bias torch.Size([1]) True
lambda_f.16.0.weight torch.Size([3, 2]) True
lambda_f.16.0.bias torch.Size([3]) True
lambda_f.16.2.weight torch.Size([3, 3]) True
lambda_f.16.2.bias torch.Size([3]) True
lambda_f.16.4.weight torch.Size([3, 3]) True
lambda_f.16.4.bias torch.Size([3]) True
lambda_f.16.6.weight torch.Size([1, 3]) True
lambda_f.16.6.bias torch.Size([1]) True
lambda_f.17.0.weight torch.Size([3, 2]) True
lambda_f.17.0.bias torch.Size([3]) True
lambda_f.17.2.weight torch.Size([3, 3]) True
lambda_f.17.2.bias t

theta_init_offset Parameter containing:
tensor([-0.1293], device='cuda:0', requires_grad=True)
rho_l1.0.0.weight Parameter containing:
tensor([[-0.3543, -0.3788,  0.0514],
        [-0.3652, -0.0944, -0.5070],
        [-0.2788, -0.3570,  0.5541]], device='cuda:0', requires_grad=True)
rho_l1.0.0.bias Parameter containing:
tensor([-0.1378, -0.4910, -0.5694], device='cuda:0', requires_grad=True)
rho_l1.0.2.weight Parameter containing:
tensor([[-0.1082, -0.0864,  0.5991],
        [ 0.6811,  0.7125,  0.3698],
        [-0.6083,  0.0364,  0.1465]], device='cuda:0', requires_grad=True)
rho_l1.0.2.bias Parameter containing:
tensor([-0.4468, -0.1658, -0.2776], device='cuda:0', requires_grad=True)
rho_l1.0.4.weight Parameter containing:
tensor([[ 0.6515,  0.8592,  0.1142],
        [ 0.2611, -0.1748,  0.2753],
        [ 0.7046,  0.5205, -0.3579]], device='cuda:0', requires_grad=True)
rho_l1.0.4.bias Parameter containing:
tensor([-0.0581,  0.0729, -0.4551], device='cuda:0', requires_grad=True)
rho_l

rho_l1.11.2.bias Parameter containing:
tensor([ 0.3351,  0.5909, -0.0933], device='cuda:0', requires_grad=True)
rho_l1.11.4.weight Parameter containing:
tensor([[-0.3095, -0.8393,  0.6966],
        [-0.3016,  0.6705, -0.6952],
        [ 0.1970,  0.5739, -0.0125]], device='cuda:0', requires_grad=True)
rho_l1.11.4.bias Parameter containing:
tensor([-0.1640,  0.3946,  0.2887], device='cuda:0', requires_grad=True)
rho_l1.11.6.weight Parameter containing:
tensor([[ 1.0222, -0.4763, -0.5979],
        [-0.9654,  0.5652,  0.5932],
        [ 0.9810, -0.4887, -0.3463]], device='cuda:0', requires_grad=True)
rho_l1.11.6.bias Parameter containing:
tensor([-0.1523, -0.0382, -0.2440], device='cuda:0', requires_grad=True)
rho_l1.11.8.weight Parameter containing:
tensor([[-0.4813,  0.7838, -0.9384],
        [-0.8986,  0.5425, -0.1742],
        [-0.3858,  0.5112, -0.7328]], device='cuda:0', requires_grad=True)
rho_l1.11.8.bias Parameter containing:
tensor([-0.0968,  0.6576,  0.4709], device='cuda:0', re

rho_l1.23.2.weight Parameter containing:
tensor([[-0.4453,  0.6725, -0.3660],
        [-0.2477, -0.2461, -0.6250],
        [-0.6804,  0.2249, -0.1918]], device='cuda:0', requires_grad=True)
rho_l1.23.2.bias Parameter containing:
tensor([-0.4550, -0.2439, -0.5151], device='cuda:0', requires_grad=True)
rho_l1.23.4.weight Parameter containing:
tensor([[-0.1150,  0.2930, -0.1775],
        [ 0.1437, -0.4381, -0.1161],
        [ 0.7540,  0.6687,  0.6378]], device='cuda:0', requires_grad=True)
rho_l1.23.4.bias Parameter containing:
tensor([0.3212, 0.8359, 0.1614], device='cuda:0', requires_grad=True)
rho_l1.23.6.weight Parameter containing:
tensor([[-0.6888, -0.4847,  0.9049],
        [-0.0390,  0.8746, -0.1578],
        [-0.3142,  0.8947, -0.7593]], device='cuda:0', requires_grad=True)
rho_l1.23.6.bias Parameter containing:
tensor([-0.1215,  0.8372,  0.4299], device='cuda:0', requires_grad=True)
rho_l1.23.8.weight Parameter containing:
tensor([[-0.4075,  0.8869,  0.7645],
        [ 0.6029, -

lambda_f.11.0.bias Parameter containing:
tensor([ 0.6491,  0.1822, -0.3722], device='cuda:0', requires_grad=True)
lambda_f.11.2.weight Parameter containing:
tensor([[-0.5295,  0.1016,  0.3032],
        [ 0.7362,  0.1217, -0.3631],
        [ 0.5597, -0.4071,  0.0568]], device='cuda:0', requires_grad=True)
lambda_f.11.2.bias Parameter containing:
tensor([-0.1045,  0.7395,  0.5436], device='cuda:0', requires_grad=True)
lambda_f.11.4.weight Parameter containing:
tensor([[ 0.5311, -0.4522, -0.8291],
        [-0.1880,  0.1932,  1.1731],
        [ 0.9164, -0.7261, -1.1046]], device='cuda:0', requires_grad=True)
lambda_f.11.4.bias Parameter containing:
tensor([-0.5314,  0.9102, -0.6607], device='cuda:0', requires_grad=True)
lambda_f.11.6.weight Parameter containing:
tensor([[-1.1122,  1.0045, -0.5499]], device='cuda:0', requires_grad=True)
lambda_f.11.6.bias Parameter containing:
tensor([0.0163], device='cuda:0', requires_grad=True)
lambda_f.12.0.weight Parameter containing:
tensor([[ 0.6884, 

TIMING check:
Running unrolled ADMM predict
CEHCKK: Total graphs =  4000


(4.179,
 8.65713,
 0.17863,
 0.08955,
 0.54383,
 0.05061,
 0.6451956927776337,
 array(1.1601645, dtype=float32),
 1.096644,
 0.014583419214124765)

In [8]:

print('model trained: Predicting on...')
# torch.save(model.state_dict(), 'Gista_model.pt')
print('****Train Data, same pred matrix, different samples****')
#glasso_predict(nmse_model, train_data, True)
# res_train = glasso_predict(model, train_data, True)
# print(res_train)
# if len(valid_data)>0:
#     print('****Valid Data****')
#     #glasso_predict(nmse_model, valid_data, True)
#     glasso_predict(model, valid_data, True)
print('****Test Data, model_NMSE: average results over different samples****')
res_test_model_NMSE = glasso_predict(model, test_data, True)
print(res_test_model_NMSE)
print('****Test Data, tag: model_SHD : average results over different samples****')
res_test_model_SHD = glasso_predict(ps_model, test_data, True)
print(res_test_model_SHD)

model trained: Predicting on...
****Train Data, same pred matrix, different samples****
****Test Data, model_NMSE: average results over different samples****
Running unrolled ADMM predict
CEHCKK: Total graphs =  64
Adjusting the minimum eigenvalue to 1, SHOULD NOT BE CALLED AFTER THE GAMMA ADDITION!
Adjusting the minimum eigenvalue to 1, SHOULD NOT BE CALLED AFTER THE GAMMA ADDITION!
Adjusting the minimum eigenvalue to 1, SHOULD NOT BE CALLED AFTER THE GAMMA ADDITION!
Structure learning Metrics
Average result over test graphs
FDRs, TPRs, FPRs, SHDs, Ts, Ps, precisions, recalls, F_betas, auprs, aucs
nan, nan, 0.08561, 0.12144, 0.01128, 0.04226, 19.21875, 6.78802, 18.93750, 3.79092, 3.56250, 9.14104, nan, nan, 0.08561, 0.12144, 0.11853, 0.12507, 0.16947, 0.09362, 0.53874, 0.05119
(3.5625, 9.14104, 0.16947, 0.09362, 0.53874, 0.05119, 1.4715729653835297, array(1.4033219, dtype=float32), 1.1230464, 0.1756373496116388)
****Test Data, tag: model_SHD : average results over different samples***

In [9]:
model_path = "M100_N20_Ktrain10_gtBA_prob0.1_bs10_L30_lr0.004.pth"
# load model
model  = threshold_NN_lambda_unrolled_model(L, rho_init, lambda_init, theta_init_offset, gamma_init, N, nF, H, USE_CUDA=USE_CUDA)
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

Plot = False
glasso_predict(model, test_data, True)


shifting to cuda
CHECK RHO INITIAL:  Parameter containing:
tensor([[-0.3799, -0.0895, -0.3191],
        [-0.0772, -0.0326, -0.2469],
        [ 0.1088,  0.3374, -0.1493]], device='cuda:0', requires_grad=True)
Running unrolled ADMM predict
CEHCKK: Total graphs =  64
Structure learning Metrics
Average result over test graphs
FDRs, TPRs, FPRs, SHDs, Ts, Ps, precisions, recalls, F_betas, auprs, aucs
0.81002, 0.08219, 0.99870, 0.01033, 0.56839, 0.24181, 96.53125, 40.09246, 18.93750, 3.79092, 115.43750, 43.11499, 0.18998, 0.08219, 0.99870, 0.01033, 0.31193, 0.10171, 0.43186, 0.08604, 0.81742, 0.05214


(115.4375,
 43.11499,
 0.43186,
 0.08604,
 0.81742,
 0.05214,
 10.987707376480103,
 array(12.553672, dtype=float32),
 26.037508,
 7.124815483485574)